In [24]:
from abc import ABCMeta, abstractmethod
from urllib.parse import urlencode
import requests
from datetime import datetime

class NCBI_Searcher(metaclass=ABCMeta):
    """ 'Interface' que define a utilização da API das databases da NCBI.
    """
    
    search_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    meta_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi'
    fetch_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
        
    def search(self, queryterms: list=None, search_type: str=None,
               start_year: int=1900, end_year: int=None,
               max_records: int=20, start_record: int=0,
               author: str=None):
        """
        Realiza uma pesquisa NCBI.
        
        @param queryterms: list of lists. Terms within the same list are
            separated by an OR. Lists are separated by an AND
        @param search_type: meta_data or querytext. 
            meta_data: This field enables a free-text search of all 
                configured metadata fields and the abstract.
            querytext: This field enables a free-text search of all 
                fields. 
        @param start_year: Start value of Publication Year to restrict results by.
        @param end_year: End value of Publication Year to restrict results by.
        @param max_records: The number of records to fetch.
        @param start_record: Sequence number of first record to fetch.
        @param author: An author's name. Searches both first name and last name
            
        @return: uma lista de títulos e IDs no formato [(title, id)]
        """
        
        term = self._search_term(queryterms, search_type=search_type)
        if author:
            term = "%s AND %s[Author]" % (term, author)
        
        fixed_payload = {"retmode": "json", "datetype": "pdat",
                         "db":self._db, "sort":self._sort_order}
        payload = {"term":term, 
                   "retmax": max_records, "retstart":start_record,
                   "mindate": start_year, "maxdate": end_year or datetime.now().year}
        payload.update(fixed_payload)
        
        url = "%s?%s" % (self.search_url, urlencode(payload))
        
        response = requests.get(url).json()['esearchresult']
        
        print ('TERMs: %s' % response['querytranslation'])
        print('QTD. resultados: %s' % response['count'])
        
        id_list = response['idlist']

        if id_list:
            result = self.get_article_metadata(*id_list)['result']

            return [(result[uid]['title'], uid) for uid in result['uids']]
        return []
    
    def get_article_metadata(self, *args):
        """
        Retorna os metadados do(s) artigo(s).
        
        @param args: IDs dos artigos. O ID do artigo depende da base de dados.
            O ID de um artigo pode ser diferente entre a PMC e a PubMed
            por exemplo. Podem ser passados vários IDs ao mesmo tempo.
        @param db: base de dados a ser pesquisada. Se for mais de uma,
            pode ser separado por vírgula. As mais úteis serão:
            pmc, pubmed
        @param retmode: A forma de retorno. None será em XML.
        
        @return: o json cru retornado pela API.
        """
        id_list = ','.join([str(x) for x in args])
            
        payload = {"id":id_list, "db":self._db, "retmode":"json"}
        url = "%s?%s" % (self.meta_url, urlencode(payload))
        
        r = requests.get(url).json()
        
        return r        
    
    def _search_term(self, queryterms: list, search_type: str=None):
        """Monta o termo de pesquisa completo para mandar para a API."""
        
        if search_type in ['querytext', None]:
            # Retorna simplesmente a busca concatenando com os OR's e AND's
            return "(%s)" % " AND ".join(["(%s)" % " OR ".join(orses)for orses in queryterms])
        elif search_type != 'meta_data':
            raise Exception('Tipo de pesquisa não faz sentido: %s\nTipos suportados:' % search_type)
        
        # Retorna concacentando com os OR'S e AND's, mas embutindo também os campos de pesquisa em cada termo
        queryterms = [[self._embutir_fields(orses) for orses in andes] for andes in queryterms]
        return "(%s)" % " AND ".join(["(%s)" % " OR ".join(orses)for orses in queryterms])
    
    def _embutir_fields(self, term: str):
        """Faz uma transformação, embutindo fields no termo de pesquisa.
        Isso é para poder realizar a pesquisa em apenas alguns campos ao invés de todos.
        
        Exemplo: sendo self.__fields = ['title', 'abstract'],
        a chamada 
        `self._embutir_fields("machine learning")`
        Transforma:
            machine learning ---> (machine learning[title] OR machine learning[abstract])
        """
        
        return "(%s)" % " OR ".join(["%s[%s]" % (term, field) for field in self._fields])
    
    @property
    @abstractmethod
    def _fields(self):
        """Cada subclasse deverá definir quais serão os campos de pesquisa de cada termo.
        O retorno deverá ser uma lista de fields.
        Exemplo: 
        return ['title', 'abstract']
        """
        pass
    
    @property
    @abstractmethod
    def _db(self):
        """Cada subclasse deverá definir o seu banco.
        Exemplo: 
        return 'pmc'
        """
        pass
    
    @property
    @abstractmethod
    def _sort_order(self):
        """Cada classe deverá definir o parâmetro sort_order.
        Exemplo:
        return 'Journal'
        """
        pass
        
    @property
    @abstractmethod
    def _article_url(self):
        """Cada classe deverá definir a URL da página de um artigo.
        """
        pass
    
class PMC_Searcher(NCBI_Searcher):
    """Realiza pesquisas na base PMC."""
    
    @property
    def _fields(self):
        return ['Abstract', 'Body - Key Terms', 'MeSH Terms',
                'MeSH Major Topic', 'Methods - Key Terms']
    
    @property
    def _db(self):
        return 'pmc'
    
    @property
    def _sort_order(self):
        return 'relevance'
    
    
class PubMed_Searcher(NCBI_Searcher):
    """Realiza pesquisas na base PubMed."""
    
    @property
    def _fields(self):
        return ['Text Words']
    
    @property
    def _db(self):
        return 'pubmed'
    
    @property
    def _sort_order(self):
        return ''
    
    @property
    def _article_url(self):
        return "https://www.ncbi.nlm.nih.gov/pubmed/"
    
    def get(self,*args):
        id_list = ','.join([str(x) for x in args])
        
        payload = {"id":id_list, "db":self._db, "retmode":"xml"}
        url = "%s?%s" % (self.fetch_url, urlencode(payload))
        
        soup = bsoup(requests.get(url).content, "xml")
        
        pmcarticles = soup.findAll('PubmedArticle')
        
        # OBS:
        # Se definir o keyvalues, pode colocar como interface!
        documentos = [{
            "resumo": art.AbstractText.text,
            "resumo_url": "%s%s" % (self._article_url, art.PMID.text)
        } for art in pmcarticles]
        
        return documentos

In [35]:
technology_queryterms = [
    'machine learning', 'deep learning', 'artificial intelligence', 
    'neural network', 'scoring system'
]

health_queryterms = [
    'coronary artery disease', 'chest pain', 'heart disease', 'MACE', 
    'Acute Cardiac Complications'
]

# technology_queryterms = [
#     'machine learning', 'deep learning'
# ]

# health_queryterms = [
#     'coronary artery disease', 'chest pain'
# ]

queryterms = [technology_queryterms, health_queryterms]

r = PMC_Searcher().search(queryterms=queryterms, start_year=2008)
[(x[0][:30],x[1]) for x in r[:2]]

TERMs: ((("machine learning"[MeSH Terms] OR ("machine"[All Fields] AND "learning"[All Fields]) OR "machine learning"[All Fields]) OR (deep[All Fields] AND ("learning"[MeSH Terms] OR "learning"[All Fields])) OR ("artificial intelligence"[MeSH Terms] OR ("artificial"[All Fields] AND "intelligence"[All Fields]) OR "artificial intelligence"[All Fields]) OR ("neural networks (computer)"[MeSH Terms] OR ("neural"[All Fields] AND "networks"[All Fields] AND "(computer)"[All Fields]) OR "neural networks (computer)"[All Fields] OR ("neural"[All Fields] AND "network"[All Fields]) OR "neural network"[All Fields]) OR (scoring[All Fields] AND system[All Fields])) AND (("coronary artery disease"[MeSH Terms] OR ("coronary"[All Fields] AND "artery"[All Fields] AND "disease"[All Fields]) OR "coronary artery disease"[All Fields]) OR ("chest pain"[MeSH Terms] OR ("chest"[All Fields] AND "pain"[All Fields]) OR "chest pain"[All Fields]) OR ("heart diseases"[MeSH Terms] OR ("heart"[All Fields] AND "diseases"[

[('ESICM LIVES 2016: part one: Mi', '5042924'),
 ('36th International Symposium o', '5493079')]

In [ ]:
r = PubMed_Searcher().search(queryterms=queryterms, start_year=2008)
[(x[0][:30],x[1]) for x in r[:2]]

In [3]:
from bs4 import BeautifulSoup as bsoup
xml = requests.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=28663072&retmode=xml")
soup = bsoup(xml.content, "xml").findAll('PubmedArticle')

In [ ]:
PubMed_Searcher().get(28663072)